In [4]:
from flaml import autogen

llm_config = {
    "request_timeout": 600,
    "seed": 41,
    "config_list": [{
        "model": "gpt-3.5-turbo-16k",
        "api_key": "",
        "temperature": 0.25,
        "frequency_penalty": 0.5,
        "max_tokens": 800,
    }],
    "temperature": 0,
}

In [10]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.memory import ConversationBufferMemory
from langchain.llms import OpenAI
from langchain.chains import ConversationalRetrievalChain

loader = [ PyPDFLoader('marketer.pdf')]
docs = [ ]
for l in loader:
    docs.extend(l.load())
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000)
docs = text_splitter.split_documents(docs)


[Document(page_content='1', metadata={'source': 'marketer.pdf', 'page': 0}), Document(page_content='2 \n  \n \n \n \n \n Как использовать более чем \n100 психологических \nсекретов  рекламного \nагентства, чтобы делать \nБОЛЬШИЕ  деньги, продавая \nчто угодно и кому угодно. \n \n \nДРЮ ЭРИК УИТМ ЕН \n \nCA$HVERTISING', metadata={'source': 'marketer.pdf', 'page': 1}), Document(page_content='3 \n  \n \n \nХотите лучших результатов от вашей \nрекламы?  Эксперт в рекламе \nДрю Эрик Уитмен говорит:  \n \n«Я научу вас,  как создавать рекламу мощнее, эффективнее и прибыльнее \nчем ваши конкуренты будут  знать всю свою карьеру –  гарантировано! » \n \nДорогой друг:  \nВы когда -нибудь видел и фокус с распиливанием прекрасной девушки \nпополам? Глотателя 28 -ми дюймового лезвия ножа? Хождение босиком по \nраскаленным углям? Эти ловкие трюки удивительны, они ошеломляют. И \nкажутся совершенно невозможными. Но правда заключается в том, что если \nвы знает е, в чем секрет, не составит большого тру

In [11]:
vector_store = Chroma(
    collection_name="full_documents",
    embeddings_function=OpenAIEmbeddings()
)
vector_store.add_documents(docs)

ValidationError: 1 validation error for OpenAIEmbeddings
__root__
  Did not find openai_api_key, please add an environment variable `OPENAI_API_KEY` which contains it, or pass  `openai_api_key` as a named parameter. (type=value_error)

In [ ]:
# Создание копирайтера как ассистента на autogen
  
assistant = autogen.AssistantAgent(
    name="copywriter",
    llm_config=llm_config,
    is_termination_msg=lambda x: True if "TERMINATE" in x.get("content") else False,
)

# Создание маркетолога как ассистента  
assistant = autogen.AssistantAgent(
    name="marketer",
    llm_config=llm_config,
    is_termination_msg=lambda x: True if "TERMINATE" in x.get("content") else False,
)

# Копирайтер должен написать текст на тему рекламы какого-то продукта, маркетолог должен проанализировать и дать список рекомендаций на основе книги в pdf, на основе него копирайтер перерабатывает текст. Так 3-4 цикла. Сравниваем первый вариант и последний.


user_proxy = autogen.UserProxyAgent(
    
    name="Admin",
    human_input_mode="NEVER",
)

# Настроить простую систему из двух автономных агентов. Один - копирайтер, другой - маркетолог, развернуть веб-интерфейс для контроля. На бэке рекомендую OpenAI GPT-4 turbo. Агенты - AutoGen. Материал для обучения маркетолога: [Как_использовать_более_чем_100_психологических_секретов_рекламного.pdf](https://prod-files-secure.s3.us-west-2.amazonaws.com/b99708e6-831d-4012-bac7-8a7c9bc63eb6/910f8be2-1e64-4397-894a-73051455f2f1/%D0%9A%D0%B0%D0%BA_%D0%B8%D1%81%D0%BF%D0%BE%D0%BB%D1%8C%D0%B7%D0%BE%D0%B2%D0%B0%D1%82%D1%8C_%D0%B1%D0%BE%D0%BB%D0%B5%D0%B5_%D1%87%D0%B5%D0%BC_100_%D0%BF%D1%81%D0%B8%D1%85%D0%BE%D0%BB%D0%BE%D0%B3%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%B8%D1%85_%D1%81%D0%B5%D0%BA%D1%80%D0%B5%D1%82%D0%BE%D0%B2_%D1%80%D0%B5%D0%BA%D0%BB%D0%B0%D0%BC%D0%BD%D0%BE%D0%B3%D0%BE.pdf) Формат передачи может быть любой - демо или видео работы. Копирайтер должен написать текст на тему рекламы какого-то продукта, маркетолог должен проанализировать и дать список рекомендаций на основе книги выше, на основе него копирайтер перерабатывает текст. Так 3-4 цикла. Сравниваем первый вариант и последний.
